<a href="https://colab.research.google.com/github/ElenaVillano/sentiment_analysis_tweets/blob/main/notebooks/limpieza_bien.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NOTEBOOK DE LIMPIEZA

In [5]:
!pip install "git+https://github.com/ElenaVillano/sentiment_analysis_tweets.git#egg=nlptweet&subdirectory=src" --quiet

In [2]:
#!pip uninstall nlptweet

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import nltk
#import re
import timeit
import string
# Nuestro paquete
import nlp


#call the nltk downloader
nltk.download('punkt')

from dateutil import parser

#from sklearn.model_selection import train_test_split

# Carga un set de stopwords predefinidas
from nltk.corpus import stopwords

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
# Nombramiento de columnas
col_names = ['target', # Polaridad del twet 0=negativo, 2=neutral, 4=positivo
             'ids',    # ID tweet
             'date',   # Fecha y hora del tweet
             'flag',   # QUERY
             'user',   # Usuario del tweet
             'text']   # Texto del tweety

# Carga y limpieza de datos

In [12]:
# Requiered to select a file to be imported into colab
# Not useful if running locally
from google.colab import files
uploaded = files.upload()

In [10]:
training =  pd.read_csv('smaller_sample_200000.csv',
                 encoding='latin-1')

In [13]:
test = pd.read_csv('testdata_manual_2009_06_14.csv', names=col_names)

In [14]:
print(training.shape)
print(test.shape)

(200000, 6)
(498, 6)


In [15]:
# Ejemplo
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,Sat Jun 20 01:09:32 PDT 2009,NO_QUERY,evilcaz,Far too tired to be at work
8,0,2233840533,Thu Jun 18 21:59:23 PDT 2009,NO_QUERY,TiffanyNicoleN,"I wish today never happened. On top of that, I..."
27,0,2254342354,Sat Jun 20 09:36:05 PDT 2009,NO_QUERY,AJMcCleary,"@ReallyShecky - Indeed, come on down. The sick..."
41,4,1794361968,Thu May 14 05:38:02 PDT 2009,NO_QUERY,cherrybomb54,Now I get to make incredibly long Tweets thank...
44,0,1686244630,Sun May 03 04:31:06 PDT 2009,NO_QUERY,anggieholics,btw whats wrong w/ being single? and if i turn...
35,4,2064953318,Sun Jun 07 07:33:41 PDT 2009,NO_QUERY,heidirenee92,@allucha It would be a good idea not to mess w...
48,4,2004632930,Tue Jun 02 08:30:15 PDT 2009,NO_QUERY,Hadari_B,At a birthday party of a friend !
155,4,1687038778,Sun May 03 07:34:54 PDT 2009,NO_QUERY,lynettebrennan,"is relaxing on the couch, Sundays are the best"


## Convertimos a minúsculas

In [16]:
from nlp.preprocessing import convierte_a_minusculas

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
training = convierte_a_minusculas(training)
test = convierte_a_minusculas(test)

In [18]:
# Ejemplo
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,"i wish today never happened. on top of that, i..."
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,"@reallyshecky - indeed, come on down. the sick..."
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single? and if i turn...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,@allucha it would be a good idea not to mess w...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend !
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,"is relaxing on the couch, sundays are the best"


## Reemplazamos las URLs con el texto "URL"

In [19]:
from nlp.preprocessing import reemplazar_urls

In [20]:
training['text'] = training['text'].map(lambda s: reemplazar_urls(s))
test['text'] = test['text'].map(lambda s: reemplazar_urls(s))

In [21]:
# Ejemplo
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,"i wish today never happened. on top of that, i..."
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,"@reallyshecky - indeed, come on down. the sick..."
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single? and if i turn...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,@allucha it would be a good idea not to mess w...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend !
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,"is relaxing on the couch, sundays are the best"


## Reemplazamos las menciones @ con la palabra "USER_MENTION"



In [22]:
from nlp.preprocessing import reemplazar_usuarios

In [23]:
training['text'] = training['text'].map(lambda s: reemplazar_usuarios(s))
test['text'] = test['text'].map(lambda s: reemplazar_usuarios(s))

In [24]:
# Ejemplo
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,"i wish today never happened. on top of that, i..."
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,"user_mention - indeed, come on down. the sick ..."
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single? and if i turn...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,user_mention it would be a good idea not to me...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend !
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,"is relaxing on the couch, sundays are the best"


## Quitamos el Hashtag , pero dejamos la palabra

In [25]:
# The regular expression used to match hashtags is #(\S+).
from nlp.preprocessing import quitar_hashtag

In [26]:
training['text'] = training['text'].map(lambda s: quitar_hashtag(s))
test['text'] = test['text'].map(lambda s: quitar_hashtag(s))

In [27]:
# Ejemplo
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,"i wish today never happened. on top of that, i..."
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,"user_mention - indeed, come on down. the sick ..."
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single? and if i turn...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,user_mention it would be a good idea not to me...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend !
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,"is relaxing on the couch, sundays are the best"


## Quitamos los Retweets, sólo la palabra RT , dejamos el comentario

In [28]:
#  The regular expression used to match retweets is \brt\b.
from nlp.preprocessing import quitar_RT

In [29]:
training['text'] = training['text'].map(lambda s: quitar_RT(s))
test['text'] = test['text'].map(lambda s: quitar_RT(s))

In [30]:
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,"i wish today never happened. on top of that, i..."
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,"user_mention - indeed, come on down. the sick ..."
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single? and if i turn...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,user_mention it would be a good idea not to me...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend !
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,"is relaxing on the couch, sundays are the best"


## Quitamos espacios o puntos extras

In [31]:
## Reemplazamos los dobles puntos (o más) con un espacio , dos o más espacios con 1 espacio y hacemos strip de espacios y comillas
# Strip any punctuation [’"?!,.():;-'] from the word ??
from nlp.preprocessing import quitar_caracteres_especiales

In [32]:
training['text'] = training['text'].map(lambda s: quitar_caracteres_especiales(s))
test['text'] = test['text'].map(lambda s: quitar_caracteres_especiales(s))

In [33]:
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,i wish today never happened on top of that i m...
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,user_mention indeed come on down the sick hea...
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single and if i turn ...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,user_mention it would be a good idea not to me...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,is relaxing on the couch sundays are the best


## Quitamos expresiones con letras repetidas

In [34]:
#Convert 2 or more letter repetitions to 2 letters.
#Some people send tweets like I am sooooo
#happpppy adding multiple characters to emphasize
#on certain words. This is done to handle such tweets
#by converting them to I am soo happy
from nlp.preprocessing import quitar_letras_repetidas

In [35]:
training['text'] = training['text'].map(lambda s: quitar_letras_repetidas(s))
test['text'] = test['text'].map(lambda s: quitar_letras_repetidas(s))

In [36]:
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,i wish today never happened on top of that i m...
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,user_mention indeed come on down the sick hea...
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single and if i turn ...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,user_mention it would be a good idea not to me...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,is relaxing on the couch sundays are the best


## Quitar caracteres nonascii

In [37]:
from nlp.preprocessing import quitar_nonascii

In [38]:
training['text'] = training['text'].map(lambda s: quitar_nonascii(s))
test['text'] = test['text'].map(lambda s: quitar_nonascii(s))

In [39]:
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,i wish today never happened on top of that i m...
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,user_mention indeed come on down the sick hea...
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single and if i turn ...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,user_mention it would be a good idea not to me...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,is relaxing on the couch sundays are the best


## Removemos stopwords

In [44]:
nltk.download('stopwords') # Use predefined list of stopwords from nltk

#STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
from nlp.preprocessing import separar_abreviaciones

In [42]:
training['text'] = training['text'].map(lambda s: separar_abreviaciones(s))
test['text'] = test['text'].map(lambda s: separar_abreviaciones(s))

In [43]:
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,i wish today never happened on top of that i m...
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,user_mention indeed come on down the sick hea...
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single and if i turn ...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,user_mention it would be a good idea not to me...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,is relaxing on the couch sundays are the best


In [45]:
from nlp.preprocessing import remove_stopwords

In [46]:
training['text'] = training['text'].map(lambda s: remove_stopwords(s))
test['atext'] = test['text'].map(lambda s: remove_stopwords(s))

In [47]:
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far tired work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,wish today never happened top miss user_mentio...
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,user_mention indeed come sick heat really quit...
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,get make incredibly long tweets thanks twitzer...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ single turn lesbian let u know
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,user_mention would good idea mess mr potatohea...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,birthday party friend
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,relaxing couch sundays best


## Stemming con NLTK

In [ ]:
training['text'] = training['text'].map(lambda s: oracion_raiz(s))
test['atext'] = test['text'].map(lambda s: oracion_raiz(s))

In [ ]:
training

,target,ids,date,flag,user,text
0,0,1467810369,mon apr 06 22:19:45 pdt 2009,no_query,_thespecialone_,user_ment url aww bummer shoulda got david car...
1,0,1467810672,mon apr 06 22:19:49 pdt 2009,no_query,scotthamilton,upset updat facebook text might cri result sch...
2,0,1467810917,mon apr 06 22:19:53 pdt 2009,no_query,mattycus,user_ment dive mani time ball manag save 50 % ...
3,0,1467811184,mon apr 06 22:19:57 pdt 2009,no_query,ellectf,whole bodi feel itchi like fire
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,user_ment behav mad see
...,...,...,...,...,...,...
1599995,4,2193601966,tue jun 16 08:40:49 pdt 2009,no_query,amandamarie1028,woke school best feel ever
1599996,4,2193601969,tue jun 16 08:40:49 pdt 2009,no_query,thewdboards,thewdbcom cool hear old walt interview url
1599997,4,2193601991,tue jun 16 08:40:49 pdt 2009,no_query,bpbabe,readi mojo makeov ask detail
1599998,4,2193602064,tue jun 16 08:40:49 pdt 2009,no_query,tinydiamondz,happi 38th birthday boo time tupac amaru shakur


Quitamos datos al entrenamiento, empecemos con 100,000 

In [ ]:
data_set_1 = training= training.head(100000)

In [ ]:
data_set_1

,target,ids,date,flag,user,text
0,0,1467810369,mon apr 06 22:19:45 pdt 2009,no_query,_thespecialone_,user_ment url aww bummer shoulda got david car...
1,0,1467810672,mon apr 06 22:19:49 pdt 2009,no_query,scotthamilton,upset updat facebook text might cri result sch...
2,0,1467810917,mon apr 06 22:19:53 pdt 2009,no_query,mattycus,user_ment dive mani time ball manag save 50 % ...
3,0,1467811184,mon apr 06 22:19:57 pdt 2009,no_query,ellectf,whole bodi feel itchi like fire
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,user_ment behav mad see
...,...,...,...,...,...,...
99995,0,1793820630,thu may 14 04:05:58 pdt 2009,no_query,babybigmouth,son develop new habit wake 530am second pot co...
99996,0,1793820812,thu may 14 04:06:00 pdt 2009,no_query,paramoreroxx,look like router broke tweet fone
99997,0,1793820858,thu may 14 04:06:00 pdt 2009,no_query,beckypope,realli dont want colleg right wish sunni
99998,0,1793820982,thu may 14 04:06:02 pdt 2009,no_query,xmissxxxmurderx,user_ment *offer pepto*


# Entrenamos un Tokenizer

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers

# Entrena un Tokenizer. Consiste en:
# Crea un diccionario numerado de las palabras existentes en el corpus, y devuelve
# por cada palabra, el número entero de su índice en el diccionario.
# En este caso, considera las N palabras más frecuentes.
# oov_token = constante asiganda para palabras fuera del vocabulario (NOT USED HERE)


max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data_set_1.text)
sequences = tokenizer.texts_to_sequences(data_set_1.text)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...  555 1577    6]
 [   0    0    0 ...   10  217  706]
 [   0    0    0 ...  357    3 2676]
 ...
 [   0    0    0 ...   60   20  369]
 [   0    0    0 ...    2    1 1207]
 [   0    0    0 ...   37   87  313]]


## Separamos en train, test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tweets, data_set_1.target.values, test_size=0.3, random_state=23042021)

In [ ]:
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (70000, 200)
y_train (70000,)
X_test (30000, 200)
y_test (30000,)


# Creamos el modelo

### RNN, tomado de: https://www.kaggle.com/smitshah00/sentiment-analysis-ml-rnn

In [ ]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint


model2 = Sequential()
model2.add(layers.Embedding(max_words, 128))
model2.add(layers.LSTM(64,dropout=0.5))
model2.add(layers.Dense(16, activation='relu'))
model2.add(layers.Dense(8, activation='relu'))
model2.add(layers.Dense(1,activation='sigmoid'))
model2.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         640000    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 16)                1040      
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 690,593
Trainable params: 690,593
Non-trainable params: 0
_________________________________________________________________


In [ ]:
checkpoint2 = ModelCheckpoint("rnn_model.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test),callbacks=[checkpoint2])

Epoch 1/10
2188/2188 [==============================] - 366s 166ms/step - loss: 0.0219 - accuracy: 0.9963 - val_loss: 2.3566e-06 - val_accuracy: 1.0000

Epoch 00001: val_accuracy improved from -inf to 1.00000, saving model to rnn_model.hdf5
Epoch 2/10
2188/2188 [==============================] - 360s 165ms/step - loss: 1.5734e-06 - accuracy: 1.0000 - val_loss: 4.3041e-07 - val_accuracy: 1.0000

Epoch 00002: val_accuracy did not improve from 1.00000
Epoch 3/10
2188/2188 [==============================] - 361s 165ms/step - loss: 3.1633e-07 - accuracy: 1.0000 - val_loss: 1.1139e-07 - val_accuracy: 1.0000

Epoch 00003: val_accuracy did not improve from 1.00000
Epoch 4/10
2188/2188 [==============================] - 360s 165ms/step - loss: 8.4086e-08 - accuracy: 1.0000 - val_loss: 3.1955e-08 - val_accuracy: 1.0000

Epoch 00004: val_accuracy did not improve from 1.00000
Epoch 5/10
2188/2188 [==============================] - 371s 170ms/step - loss: 2.4389e-08 - accuracy: 1.0000 - val_loss: 9

# Probamos

In [ ]:
model = keras.models.load_model('rnn_model.hdf5')
sequence = tokenizer.texts_to_sequences(['this data science article is the best ever'])
test = pad_sequences(sequence, maxlen=max_len)
pred = model.predict(test)
if pred > 0.5:
  print('Positive')
else:
  print('Negative')